In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install numpy matplotlib scikit-learn ipython

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import FastICA
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from WDEN import wden

In [ ]:
plt.rcParams.update({"font.size": 12, 'font.weight': 'bold', "lines.linewidth": 2})

In [ ]:
DATA_DIR = './data/'
OUTPUT_DIR = './output/'

##### Load the data

In [ ]:
# Define path and load data
# data = pd.read_csv(data_path+'All_spectra.csv', skiprows=2)
data = pd.read_csv(f"{DATA_DIR}560 spectra.csv")
cols = data.columns

# Find indexvalues for range 500-2000
# ind1, ind2 = (np.where(np.array(wav_num).astype(int)==500)[0])[0], (np.where(np.array(wav_num).astype(int)==2000)[0])[0]
# wavenumber = wav_num[ind2:ind1]
# data = data.iloc[ind2:ind1, 1:]

data = data[data.Wavenumber>=500]
data = data[data.Wavenumber<=2000]
wavenumber = data["Wavenumber"]
data = data.drop(columns=["Wavenumber"])
# print(data.head())

# Perform minmaxscalar normalization
data_minmax = (data-data.min())/(data.max()-data.min())


In [ ]:
# fig, ax = plt.subplots(1, len(cf),figsize=(16, 2), layout='constrained')
# for i in range(len(cf)):
#     ax[i].plot(cf[i], c='r', alpha=0.5, lw=3, label='Noisy')
#     ax[i].plot(cf_thr[i], c='k', alpha=0.7, label='Thresholded')
#     # ax[i].legend(frameon=False)
# plt.show()

In [ ]:
# Plot the mean of the data and peak region for parafin
plt.figure(figsize=(8, 4))
plt.plot(wavenumber, data_minmax.mean(axis=1), 'b')
# plt.plot(wavenumber, np.mean(data_den, axis=1), 'r')
plt.xlim([wavenumber.iloc[-1], wavenumber.iloc[0]])
plt.axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
plt.axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
plt.axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='k', alpha=0.5)
plt.axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Define ICA and obtain the components
ncomp = 9
ica = FastICA(n_components=ncomp, random_state=42)
ica_comp = ica.fit_transform(data_minmax)
mix_mat = ica.mixing_
print(ica_comp.shape, mix_mat.shape)

In [ ]:
c = 3
if ncomp % c == 0:
    r = ncomp//c
else:
    r = ncomp//c + 1

fig, ax = plt.subplots(r, c, figsize=(6*c, 4*r), layout='constrained')
i, j = 0, 0
while i*c + j < ncomp:
    # ax[i, j].plot(wavenumber, signal[:, i], 'b', label='Comp-%s'%i)
    ax[i, j].plot(wavenumber, np.abs(ica_comp[:, i]), 'b', label='Comp-%s'%(i*c + j))
    ax[i, j].axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
    ax[i, j].axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
    ax[i, j].axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='r', alpha=0.5)
    ax[i, j].axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)
    ax[i, j].set_xlim(wavenumber.iloc[-1], wavenumber.iloc[0])
    ax[i, j].set_ylim(0, np.max(np.abs(ica_comp)))
    ax[i, j].legend(frameon=False)
    # display.clear_output(wait=True)
    # plt.pause(5)
    
    j += 1
    if j == c:
        j = 0
        i += 1
        

plt.tight_layout()
plt.show()

In [ ]:
# Partial least square regression to regress out wax spectra
# clmn = [1, 3, 4, 6, 8, 9, 11]
# clmn = [0, 4, 6, 7, 8]
clmn = [2, 3, 6, 8]
wax_comp = np.zeros((len(ica_comp[:, 1]), len(clmn)), dtype=ica_comp.dtype)
for i in range(len(clmn)):
    wax_comp[:, i] = ica_comp[:, clmn[i]]
# wax_comp = ica_comp[:, 1]#.reshape(-1, 1)
# wax_comp = np.vstack((ica_comp[:, 1], ica_comp[:, 3], ica_comp[:, 7], ica_comp[:, 9], ica_comp[:, 13])).T#.reshape(-1, 1)

pls = PLSRegression(n_components=wax_comp.shape[1], max_iter=5000)
pls.fit(wax_comp, ica_comp)
wax_est = pls.predict(wax_comp)
print(wax_est.shape)

# Subtract the wax signal from the original spectra
correct_signal = ica_comp - wax_est
correct_spectra = np.dot(correct_signal, mix_mat.T)
# print(wax_comp.shape, correct_spectra.shape)

In [ ]:

fig, ax = plt.subplots(3, 1, figsize=(8, 12), layout='constrained')
ax[0].plot(wavenumber, np.mean(data_minmax, axis=1), 'b', lw=1, label='Original')
ax[0].plot(wavenumber, np.mean(correct_spectra, axis=1), 'r', lw=1, label='Corrected')

ax[1].plot(wavenumber, np.abs(np.abs(np.mean(correct_spectra, axis=1))), 'r', lw=1, label='Corrected')

ax[2].plot(wavenumber, np.mean(data_minmax, axis=1)-np.mean(correct_spectra, axis=1), 'b', lw=1, label='Wax')

for i in range(3):
    ax[i].axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
    ax[i].axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
    ax[i].axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='k', alpha=0.5)
    ax[i].axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)
    ax[i].set_xlim([wavenumber.iloc[-1], wavenumber.iloc[0]])
    ax[i].set_xlabel('Wavenumber', size=14, weight='bold')
    ax[i].legend(frameon=False)
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 12), layout='constrained')
ax[0].plot(wavenumber, np.mean(data_minmax, axis=1), 'b', lw=1, label='Original')
ax[0].plot(wavenumber, np.mean(correct_spectra, axis=1), 'r', lw=1, label='Corrected')

ax[1].plot(wavenumber, np.mean(correct_spectra, axis=1), 'r', lw=1, label='Corrected')

ax[2].plot(wavenumber, np.mean(data_minmax, axis=1)-np.mean(correct_spectra, axis=1), 'b', lw=1, label='Wax')

for i in range(3):
    ax[i].axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
    ax[i].axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
    ax[i].axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='k', alpha=0.5)
    ax[i].axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)
    ax[i].set_xlim([wavenumber.iloc[-1], wavenumber.iloc[0]])
    ax[i].set_xlabel('Wavenumber', size=14, weight='bold')
    ax[i].legend(frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(dpi=100)
plt.plot(wavenumber, np.array(data_minmax.iloc[:, 0])-correct_spectra[:, 0], 'b', label='Wax')
# plt.plot(wavenumber, correct_spectra[:, 0], 'r', label='Corrected')
plt.xlim([wavenumber.iloc[-1], wavenumber.iloc[0]])
plt.axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
plt.axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
plt.axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='r', alpha=0.5)
plt.axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)

plt.legend(frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# Denoise data
data_den = np.zeros_like((data))
for i in range(data.shape[1]):
    cf, cf_thr, xd = wden(x = np.array(correct_spectra[:, i]), tsr = 'heursure', sorh = 's', scl = 'mln', N = 2, wname = 'coif3')
    data_den[:, i] = xd


corrected_den = pd.DataFrame(data=data_den, columns=cols[1:])

# Perform minmaxscalar normalization
corrected_minmax = (corrected_den - corrected_den.min())/(corrected_den.max() - corrected_den.min())

fig, ax = plt.subplots(2, 1, figsize=(8, 12), layout='constrained')
# ax[0].plot(wavenumber, np.mean(data_minmax, axis=1), 'b', lw=1, label='Original')
ax[0].plot(wavenumber, np.mean(correct_spectra, axis=1), 'y', lw=1, label='Corrected')
ax[0].plot(wavenumber, np.mean(corrected_den, axis=1), 'r', lw=1, label='Corrected-Denoised')

ax[1].plot(wavenumber, np.abs(np.abs(np.mean(corrected_minmax, axis=1))), 'r', lw=1, label='Corrected-Denoised-MinMax')

for i in range(2):
    ax[i].axvspan(xmin=1051, xmax=1069, ymin=0, ymax=1, color='m', alpha=0.5)
    ax[i].axvspan(xmin=1125, xmax=1180, ymin=0, ymax=1, color='y', alpha=0.5)
    ax[i].axvspan(xmin=1290, xmax=1300, ymin=0, ymax=1, color='k', alpha=0.5)
    ax[i].axvspan(xmin=1430, xmax=1480, ymin=0, ymax=1, color='g', alpha=0.5)
    ax[i].set_xlim([wavenumber.iloc[-1], wavenumber.iloc[0]])
    ax[i].set_xlabel('Wavenumber', size=14, weight='bold')
    ax[i].legend(frameon=False)
plt.show()

corrected_den.insert(0, cols[0], wavenumber.values)
corrected_minmax.insert(0, cols[0], wavenumber.values)

In [ ]:
corrected_den.to_csv(f"{DATA_DIR}corrected_spectra_denoised.csv", index=False)
corrected_minmax.to_csv(f"{DATA_DIR}corrected_spectra_denoised_minmax.csv", index=False)
